# **Semiparametric SVM training using subgradients in Spark **

#### bla, bla, bla. 

#### We will benchmark the algorithms with data files from UCI:

* **Ripley**, the Ripley dataset
* **Kwok**, the Kwok dataset
* **Twonorm**, the Twonorm dataset
* **Waveform**, the Waveform dataset
* **Covertype**, the Covertype dataset


In [6]:
import scipy.io
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.mllib.clustering import GaussianMixture
from numpy import array
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
from pyspark.mllib.regression import LabeledPoint
from sklearn.metrics import roc_curve, auc, roc_auc_score
from time import time


############## Mover a svm_utils
def build_X1(x):
    NI = len(x.features)
    k = np.vstack((np.array(1).reshape((1, 1)), x.features.reshape((NI, 1))))
    return LabeledPoint(x.label, k.T)

def load_data(kdataset):
    if kdataset == 1:
        mat = scipy.io.loadmat('./data/ripley_5fold.mat')
        name_dataset = 'Ripley'
        sigma= 1.414214
        sigma = 1.030962
        C = 100.0
        NC = 10

    if kdataset == 2:
        mat = scipy.io.loadmat('./data/kwok_5fold.mat')
        name_dataset = 'Kwok'
        sigma = 1.414214
        sigma = 0.485075
        C = 500.0
        NC = 25

    if kdataset == 3:
        mat = scipy.io.loadmat('./data/twonorm_5fold.mat')
        name_dataset = 'Twonorm'
        sigma = 7.727851
        sigma = 0.559017
        C = 1.0
        NC = 75

    if kdataset == 4:
        mat = scipy.io.loadmat('./data/waveform_5fold.mat')
        name_dataset = 'Waveform'
        sigma = 3.340698
        C = 100.0
        NC = 75
              
    index_tr = mat['index_tr']
    index_val = mat['index_val']
    x_tr =  mat['x_tr']
    x_tst = mat['x_tst']
    y_tr =  mat['y_tr']
    y_tst =  mat['y_tst']
    return x_tr, y_tr, x_tst, y_tst, sigma, C, NC, name_dataset

def get_inc_w(x, w, landa):
    k = x.features
    NI = len(k)
    k = k.reshape(NI, 1)
    ytr = x.label
    yKw = ytr * np.dot(k.T, w)
    if yKw < 1:
        incw = -landa * w + ytr * k
    else:
        incw = -landa * w
    return incw


def train_linearSVM(Xtr1RDD, NI, C, eta, landa, Niter, Samplefraction):
    w = np.zeros(NI + 1).reshape((NI + 1, 1))
    N_added = 0
    for k in range(0, Niter):
        print k,
        Xtr1RDDsampled = Xtr1RDD.sample(withReplacement=True, fraction=Samplefraction)
        incwRDD = Xtr1RDDsampled.map(lambda x: get_inc_w(x, w, landa))
        Nsample = incwRDD.count()
        N_added += Nsample
        eta = C / N_added
        w += eta * incwRDD.reduce(lambda x, y: x + y)
    return w


def in_margin(x, w):
    k = x.features
    ytr = x.label
    yKw = ytr * np.dot(k, w)
    return yKw < 1


def plot_linear_SVM(xtr, ytr, w, c, name_dataset):
    
    index1labels = np.where(ytr > 0)
    index0labels = np.where(ytr <= 0)

    plt.plot(xtr[index1labels, 0], xtr[index1labels, 1], 'b.')
    plt.plot(xtr[index0labels, 0], xtr[index0labels, 1], 'r.')
    NP = 100.0
    SUP = np.zeros((NP, NP))
    ejex = np.arange(NP).astype(float) / NP * 4.0 - 2.0
    X_mesh = np.array([0, 0, 0]).reshape((1,3))
    for m in range(0, int(NP)):
        for n in range(0, int(NP)):
            aux = np.array([1.0, ejex[m], ejex[n]]).reshape((1,3))
            X_mesh = np.vstack((X_mesh, aux))

    y_pred = np.dot(X_mesh, w)
    y_pred = y_pred[1:]
    Z = y_pred.reshape((NP,NP))
    levels = np.arange(-1.0, 2.0 , 1.0)

    CS = plt.contour(ejex, ejex, Z.T, levels, colors = 'k')
    plt.clabel(CS, fontsize=9, inline=1)
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.title(name_dataset)
    plt.grid(True)
    #plt.savefig("test.png")
    plt.plot(c[:, 0], c[:, 1], 'ro')
    plt.show()

def plot_hybrid_SVM(xtr, ytr, w, c, name_dataset):
    index1labels = np.where(ytr > 0)
    index0labels = np.where(ytr <= 0)
    plt.plot(xtr[index1labels,0], xtr[index1labels,1],'b.')
    plt.plot(xtr[index0labels,0], xtr[index0labels,1],'r.')
    NI_a = xtr.shape[1]
    NI_b = c.shape[0]
    NI = NI_a + NI_b + 1
    NP = 100.0
    SUP = np.zeros((NP,NP))
    ejex = np.arange(NP).astype(float) / NP * 4.0 - 2.0
    X_mesh = np.zeros(NI).reshape((1,NI))
    
    NC = c.shape[0]
    for m in range(0, int(NP)):
        for n in range(0, int(NP)):
            x1 = np.array([1.0, ejex[m], ejex[n]]).reshape((1,3))
            X = np.kron(np.ones(NC).reshape((NC,1)),[ejex[m], ejex[n]])
            e = X-c
            e = e**2
            e = e.sum(axis=1)
            k = np.exp(-0.5 * e /sigma/sigma)
            k = k.reshape((1, NC))
            kx1 = np.hstack((x1,k))
            X_mesh = np.vstack((X_mesh, kx1))

    y_pred = np.dot(X_mesh, w)
    y_pred = y_pred[1:]
    Z = y_pred.reshape((NP,NP))
    levels = np.arange(-1.0, 2.0 , 1.0)
    CS = plt.contour(ejex, ejex, Z.T, levels, colors = 'k')
    plt.clabel(CS, fontsize=9, inline=1)
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.title(name_dataset)
    plt.grid(True)
    plt.show()

    
def kernelG(x,c,sigma):
    NC = c.shape[0]
    X = np.kron(np.ones(NC).reshape((NC,1)),x.features)
    e = X-c
    e = e**2
    e = e.sum(axis=1)
    k = np.exp(-0.5 * e /sigma/sigma)
    k = k.reshape((NC, 1))
    return k

def build_k(x, c, sigma):
    NI = len(x.features)
    k = np.vstack((np.array(1).reshape((1,1)),x.features.reshape((NI, 1))))
    k = np.vstack((k,kernelG(x,c,sigma)))
    x = LabeledPoint(x.label, k.T)
    return x


def train_nonlinearSVM(KtrRDD, C, landa, Niter, Samplefraction):

    x = KtrRDD.take(1)[0]
    NI = len(x.features)
    w = np.zeros(NI).reshape((NI,1))
    N_added = 0
    #KtrRDDsampled = KtrRDD.sample(withReplacement=True, fraction=Samplefraction)

    for k in range(0,Niter):
        print k, 
        KtrRDDsampled = KtrRDD.sample(withReplacement=True, fraction=Samplefraction)
        incwRDD = KtrRDDsampled.map(lambda x: get_inc_w(x, w, landa))
        Nsample = incwRDD.count()
        N_added += Nsample 
        eta = C/N_added
        w += eta * incwRDD.reduce(lambda x, y: x + y)
    print "Fin!"
    return w


def predict(x, w):
    k = x.features.reshape((1,len(x.features)))
    y_pred = np.dot(k, w)
    return y_pred

def plot_ROC(Ytr, Ytst):
    fpr_tr, tpr_tr, th_tr = roc_curve(np.array(Ytr)[:,0], np.array(Ytr)[:,1])
    auc_tr = auc(fpr_tr, tpr_tr)
    #plt.plot(fpr_tr, tpr_tr,'r')

    fpr_tst, tpr_tst, th_tst = roc_curve(np.array(Ytst)[:,0], np.array(Ytst)[:,1])
    auc_tst = auc(fpr_tst, tpr_tst)
    #plt.plot(fpr_tst, tpr_tst,'g')

    '''
    plt.xlabel('fpr')
    plt.ylabel('tpr')
    plt.title(name_dataset)
    plt.grid(True)
    plt.show()
    '''
    return auc_tst

def train_hybridSVM(XtrRDD, XtstRDD, sigma, C, NC, name_dataset, Niter, Samplefraction):
    
    time_ini = time()
    eta = C
    landa = 1.0/C
    NPtr = XtrRDD.count()
    NPtst = XtstRDD.count()
    NI = len(XtrRDD.take(1)[0].features)
    XtrRDD.cache()
    XtstRDD.cache()
    T = NPtr
    
    #print NPtr, NPtst, NI

    print "Training the linear SVM model with %d iterations" % Niter

    Xtr1RDD = XtrRDD.map(lambda x: build_X1(x)).cache()
    Xtst1RDD = XtstRDD.map(lambda x: build_X1(x)).cache()

    w = train_linearSVM(Xtr1RDD, NI, C, eta, landa, Niter, Samplefraction)

    print "Done!"

    xtr = np.array(XtrRDD.map(lambda x: x.features).collect())
    ytr = np.array(XtrRDD.map(lambda x: x.label).collect())

    print "Clustering SVs"
    SV_RDD = Xtr1RDD.filter(lambda x: in_margin(x, w))
    clusters = KMeans.train(SV_RDD.map(lambda x: x.features[1:len(x.features)]), NC, maxIterations=10,runs=10, initializationMode="random")
    c = np.array(clusters.centers)

    '''
    if kdataset == 1 or kdataset == 2:   # el resto no se pueden pintar
        #SVM.plot_linear_SVM(xtr, ytr, w, c)
        plot_linear_SVM(xtr, ytr, w, c, name_dataset)
    '''
    
    print "Building the kernel expansion..."    
    KtrRDD = XtrRDD.map(lambda x: build_k(x, c, sigma)).cache()
    KtstRDD = XtstRDD.map(lambda x: build_k(x, c, sigma)).cache()

    print "Training the hybrid SVM model with %d iterations" % Niter

    w = train_nonlinearSVM(KtrRDD, C, landa, Niter, Samplefraction)

    xtr = np.array(XtrRDD.map(lambda x: x.features).collect())
    ytr = np.array(XtrRDD.map(lambda x: x.label).collect())
    '''
    if kdataset == 1 or kdataset == 2:   # el resto no se pueden pintar 
        plot_hybrid_SVM(xtr, ytr, w, c, name_dataset)
    '''
    print "Predicting and evaluating"
    y_pred_trRDD = KtrRDD.map(lambda x: (x.label, predict(x, w)[0][0]))
    y_pred_tstRDD = KtstRDD.map(lambda x: (x.label, predict(x, w)[0][0]))

    Ytr = y_pred_trRDD.collect()
    Ytst = y_pred_tstRDD.collect()

    auc_tst = plot_ROC(Ytr, Ytst)
    elapsed_time = time() - time_ini
    print "AUC = %f" % auc_tst
    print "Elapsed_time = %f" % elapsed_time
    return auc_tst, elapsed_time



###################################################################################################################

# definir variable de usuario y ejecutar condicionalmente cualquier código que dependa del contexto de ejecución

user = 'navia' # navia, roberto
modelo = 'hybrid'

if user == 'roberto':
    # definir sc
    import findspark
    findspark.init()
    from pyspark import SparkConf, SparkContext
    conf = (SparkConf().setMaster("local[4]").setAppName("My app").set("spark.executor.memory", "2g"))
    sc = SparkContext(conf = conf)

if user == 'navia':
    #sc.addPyFile("/export/usuarios01/navia/spark/SVM_spark/common/lib/svm_utils.py")
    #sc.addPyFile("/export/usuarios01/navia/spark/SVM_spark/common/lib/quadtree_utils.py")   
    #import svm_utils as SVM
    #import quadtree_utils as QUADTREE
    import numpy as np
    from pyspark.mllib.regression import LabeledPoint
    import pickle
    %matplotlib inline

kdatasets = [1, 2, 3, 4]
kfold = 1
Niter = 150
#Niter = 2
Nnodes = 32
Samplefraction = 0.05

NCs = [5, 10, 25, 50, 100, 200]

for kdataset in kdatasets:
    aucs = []
    times = []
    x_tr, y_tr, x_tst, y_tst, sigma, C, NC, name_dataset = load_data(kdataset)
    XtrRDD = sc.parallelize(np.hstack((y_tr, x_tr)), Nnodes).map(lambda x: LabeledPoint(x[0], x[1:len(x)]))
    XtstRDD = sc.parallelize(np.hstack((y_tst, x_tst)), Nnodes).map(lambda x: LabeledPoint(x[0], x[1:len(x)]))
    for NC in NCs:
        if modelo == 'hybrid':
            auc_tst, elapsed_time = train_hybridSVM(XtrRDD, XtstRDD, sigma, C, NC, name_dataset, Niter, Samplefraction)
        aucs.append(auc_tst)
        times.append(elapsed_time)
    print aucs
    print times
    filename = 'results_dataset_' + str(kdataset) + '_' + modelo + '.pkl'
    with open(filename, 'w') as f:
        pickle.dump([aucs, times], f)

Training the linear SVM model with 150 iterations
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 Done!
Clustering SVs
Building the kernel expansion...
Training the hybrid SVM model with 150 iterations
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 11